In [ ]:
# IMPORTS
from torch import optim
from tqdm.notebook import tqdm, trange
from Build_Histogram import *
from Detect_Feature_And_KeyPoints import *
from Load_Dataset_Folder import *
from Features_Processing import *
from Linear_Processsing_Pipeline import *
from Training_Poly_Processing_Pipeline import *
from Testing_Poly_Processing_Pipeline import *
from calculate_accuracy import *
from train import *
from evaluate import *
from LeNet_Implementation import *
from sklearn.model_selection import train_test_split
import torch.utils.data as data
from VGG_Pretrain import *

## Custom CNN

In [ ]:
user= 'perso'
root_path = "C:\\Users\\" + user + "\\Documents\\GitHub\\Shark-Species-Classification"
data_path = os.path.join(root_path, 'Genus Carcharhinus')


dataset = ImageLoader(data_path)

train_dataset, test_dataset = Dataset_Splitter(.5, dataset)
train_train_dataset, validation_dataset = Dataset_Splitter(.9, train_dataset)

In [ ]:
BATCH_SIZE = 64

train_iterator = data.DataLoader(train_train_dataset,
                                 shuffle=True,
                                 batch_size=BATCH_SIZE)

valid_iterator = data.DataLoader(validation_dataset,
                                 batch_size=BATCH_SIZE)

test_iterator = data.DataLoader(test_dataset,
                                batch_size=BATCH_SIZE)

In [12]:
OUTPUT_DIM = 9
model = AlexNet(OUTPUT_DIM)

NameError: name 'AlexNet' is not defined

In [13]:
criterion = nn.CrossEntropyLoss()
device = torch.device('cuda') #if torch.cuda.is_available() else 'cpu')
optimizer = optim.Adam(model.parameters())
model = model.to(device)
criterion = criterion.to(device)

EPOCHS = [1,3,5,10,15,20,30]
EPOCHS = 7

best_valid_loss = float('inf')

for epoch in trange(EPOCHS, desc="Epochs"):

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, device)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'MMnet-model.pt')

    print(f'Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

NameError: name 'model' is not defined

## VGG Pretrained

In [17]:
vgg11_config = [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M']
vgg11_layers = get_vgg_layers(vgg11_config, batch_norm=True)

OUTPUT_DIM = 9
model = VGG(vgg11_layers, OUTPUT_DIM)

In [18]:
import torchvision.models as models

pretrained_model = models.vgg11_bn(pretrained=True)

IN_FEATURES = pretrained_model.classifier[-1].in_features
final_fc = nn.Linear(IN_FEATURES, OUTPUT_DIM)

pretrained_model.classifier[-1] = final_fc




In [19]:
model.load_state_dict(pretrained_model.state_dict())

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')


The model has 128,808,713 trainable parameters


In [20]:
BATCH_SIZE = 128



LR = 1e-6
params = [
          {'params': model.features.parameters(), 'lr': LR / 10},
          {'params': model.classifier.parameters()}
         ]

optimizer = optim.Adam(params, lr=LR)

device = torch.device('cpu')

criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

In [21]:
def train(model, iterator, optimizer, criterion, device):

    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for (x, y) in tqdm(iterator, desc="Training", leave=False):

        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()

        y_pred, _ = model(x)

        loss = criterion(y_pred, y)

        acc = calculate_accuracy(y_pred, y)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)


In [22]:

def evaluate(model, iterator, criterion, device):

    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():

        for (x, y) in tqdm(iterator, desc="Evaluating", leave=False):

            x = x.to(device)
            y = y.to(device)

            y_pred, _ = model(x)

            loss = criterion(y_pred, y)

            acc = calculate_accuracy(y_pred, y)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)


In [25]:

EPOCHS = [1,3,5,7,10,15,20]

best_valid_loss = float('inf')

for epoch_ in EPOCHS:
    print("Epoch:", epoch_)
    for epoch in trange(epoch_, desc="Epochs"):


        train_loss, train_acc = train(model, train_iterator, optimizer, criterion, device)
        valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)

        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), 'VGG-model.pt')


        print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
        print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')


Epoch: 1


Epochs:   0%|          | 0/1 [00:00<?, ?it/s]

Training:   0%|          | 0/2 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

	Train Loss: 2.173 | Train Acc: 16.49%
	 Val. Loss: 2.276 |  Val. Acc: 8.33%
Epoch: 3


...

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

	Train Loss: 1.864 | Train Acc: 43.50%
	 Val. Loss: 1.900 |  Val. Acc: 50.00%


Training:   0%|          | 0/2 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

	Train Loss: 1.818 | Train Acc: 50.53%
	 Val. Loss: 1.894 |  Val. Acc: 50.00%


In [27]:
model.load_state_dict(torch.load('VGG-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion, device)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')


Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

Test Loss: 2.017 | Test Acc: 37.66%


## SVC & KMeans

In [ ]:

image_files, labels = load_dataset_folder(data_path)
features, processed_labels = Features_Processing(image_files, labels)


features_train, features_test, labels_train, labels_test = train_test_split(features, processed_labels, test_size = .2, random_state = 0)
features_train_train, features_validation, labels_train_train, labels_validation = train_test_split(features_train, labels_train, test_size = .25, random_state = 0)


c, d = Poly_Processing_Pipeline(features_train_train, features_validation, labels_train_train, labels_validation)


training_accuracy = Testing_Poly_SVC(features_train_train, features_validation, labels_train_train, labels_validation, c, d)
testing_accuracy = Testing_Poly_SVC(features_train, features_test, labels_train, labels_test,c, d)

print("c: ", c)
print("d: ", d)
print("Training Accuracy: ", training_accuracy)
print("Testing Accuracy: ", testing_accuracy)
